In [1]:
import numpy as np
import pandas as pd
import re
from collections import Counter
import matplotlib.pylab as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import os
import formula_reader

read the flow using formula_reader.standard_flow()


In [2]:
def cleaning_the_dataset(dataset_material):
    for ii in dataset_material.keys():
        if (ii.find('Unnamed') != -1):
            del dataset_material[ii]
        elif list(set(pd.isnull(dataset_material[ii].values)))[0]==True:
            del dataset_material[ii]
        else:
            pass

In [3]:
dataset_material = pd.read_excel('Data_Ghiringhelli.xlsx', 'Material Data')
dataset_atomic = pd.read_excel('Data_Ghiringhelli.xlsx',   'Atomic Data')
dataset_atomic.columns = ['Z', 'A', 'IP', 'EA', 'HOMO', 'LUMO', 'rs', 'rp', 'rd', '1D']
dataset_test = pd.read_csv('dataset_test.csv')
dataset_test_new = pd.read_csv('dataset_test_new.csv')

In [4]:
cleaning_the_dataset(dataset_atomic)
cleaning_the_dataset(dataset_material)

In [5]:
column_name = formula_reader.columns_name(dataset_atomic_columns=dataset_atomic.keys()[2:-1], 
                           num_of_element=['A','B'])

In [6]:
equation_list = ['(rp_B**3 - exp(rs_B))/((rp_A**2))', 
                 '(sqrt(fabs(EA_A))+sqrt(fabs(IP_B)))/((rp_A**2))']

In [7]:
df_form = pd.read_excel('Final wrap up.xlsx', sheet_name='ZincBlend 1D')

all_num,all_equations,all_avg_mse = [],[],[]
for str_lst in df_form['OAD_gen_1 RMSE']:
    try :
        lindex = str_lst.index('(')
        rindex = str_lst.rindex(')')
        equation=str_lst[lindex:rindex+1]
        all_equations.append(equation)
    except:
        pass

In [8]:
new_equation_list = []
old_rmse, old_r2, old_cls   =[],[],[]
new_rmse, new_r2, new_cls   =[],[],[]
coeff_list                  =[]
min_slops, min_intercepts   =[],[]
r2_scores, cls_acss         =[],[]
regression = 'On'
rounding   = 12
target_property = 'r2_score' # 'RMSE' or 'classification_accuracy'

actute_species = ['RS' if i < 0 else 'ZB' for i in dataset_test.DE]
def target_func(pred, actu):
    rmse = np.round(np.sqrt(mean_squared_error(pred, actu)),rounding)
    r2 = r2_score(pred, dataset_test.DE)
    pred_species = ['RS' if i < 0 else 'ZB' for i in pred]
    quant_res    = ([i for i, j in zip(actute_species, pred_species) if i == j])
    cls_acs      = (len(quant_res)*100/82)
    return rmse, r2, cls_acs

def args_ind(target_property, temp_rmse, temp_cls_acs, temp_r2):
    if target_property == 'RMSE':
        args = np.argmin(temp_rmse)

    elif target_property == 'classification_accuracy':
        args = np.argmax(temp_cls_acs)

    elif target_property == 'r2_score':
        args = np.argmax(temp_r2)
    
    return args
        
for equation in all_equations[21:32]+all_equations[34:38]+all_equations[40:41]:
    slops, intercepts = [],[]
    equation = equation.replace('HOMOKS','HOMO')
    equation = equation.replace('LUMOKS','LUMO')
    eq_obj = formula_reader.equation_reader(equation_string=equation, 
                                            dataset=dataset_test)
    feature_array = np.array(eq_obj.run_equation_reader())
    feature_array = np.reshape(feature_array,(-1,1))
    a,b,c,d = eq_obj.get_individual_element()
    formula = eq_obj.get_formula()
    new_equation_list.append(formula)
    actu = dataset_test['DE'].values

    regressor = LinearRegression()
    regressor.fit(feature_array, actu)
    
    pred = regressor.predict(np.array(feature_array))
    rmse, r2, cls_acs = target_func(pred=pred, actu=actu)
    old_rmse.append(target_func(pred=pred, actu=actu)[0])
    old_r2.append(target_func(pred=pred, actu=actu)[1])
    old_cls.append(target_func(pred=pred, actu=actu)[2])
    print('rmse: ',old_rmse[-1], 'r2 score: ', old_r2[-1], 
          'class accuracy: ',old_cls[-1])
    coeff = regressor.coef_
    intercept = regressor.intercept_
    
    temp_rmse, value_lst, temp_r2, temp_cls_acs = [],[],[],[]
    for ii in np.arange(-1.1,1.1,0.1):
        for jj in np.arange(-1.1,1.1,0.1):
            for kk in np.arange(-1.1,1.1,0.1):
                for nn in np.arange(-1.1,1.1,0.1):
                    if (np.round(ii,4)==0. and np.round(jj,4)==0.) or \
                        (np.round(kk,4)==0. and np.round(nn,4)==0.):
                        pass
                    else:
                        x = (np.round(ii,4)*a + np.round(jj,4)*b)/ \
                            (np.round(kk,4)*c + np.round(nn,4)*d)
                        if regression == 'On':
                            try:
                                regressor = LinearRegression()
                                regressor.fit(np.array(x).reshape(-1,1), dataset_test.DE.values)
                                pred = regressor.predict(np.array(x).reshape(-1,1))
                                rmse, r2, cls_acs = target_func(pred=pred, actu=actu)
                                temp_rmse.append(rmse)
                                temp_r2.append(r2)
                                temp_cls_acs.append(cls_acs)
                                value_lst.append([np.round(ii,4), np.round(jj,4), 
                                                  np.round(kk,4), np.round(nn,4)])
                                slops.append(regressor.coef_)
                                intercepts.append(regressor.intercept_)
                            except:
                                pass
                        else:
                            pred = (coeff*x)+intercept
                            rmse, r2, cls_acs = target_func(pred=pred, actu=actu)
                            temp_rmse.append(rmse)
                            temp_r2.append(r2)
                            temp_cls_acs.append(cls_acs)
                            value_lst.append([np.round(ii,4), np.round(jj,4), 
                                              np.round(kk,4), np.round(nn,4)])
        args_ind(target_property, temp_rmse, temp_cls_acs, temp_r2)
    
    args = args_ind(target_property, temp_rmse, temp_cls_acs, temp_r2)
    print(value_lst[args], temp_rmse[args], temp_r2[args], temp_cls_acs[args])
    diff=  [0.1,0.01,0.001]
    interval = [0.02,0.002,0.0002]
    for ll in range(len(diff)):
        args = args_ind(target_property, temp_rmse, temp_cls_acs, temp_r2)
        range_lst = value_lst[args]
        for ii in (np.arange(range_lst[0]-diff[ll],
                             range_lst[0]+diff[ll],interval[ll])):
            for jj in (np.arange(range_lst[1]-diff[ll], 
                                 range_lst[1]+diff[ll],interval[ll])):
                for kk in (np.arange(range_lst[2]-diff[ll], 
                                 range_lst[2]+diff[ll],interval[ll])):
                    for nn in (np.arange(range_lst[3]-diff[ll], 
                                 range_lst[3]+diff[ll],interval[ll])):
                        if np.round(ii,4)==0. or np.round(jj,4)==0.:
                            pass
                        else:
                            x = (np.round(ii,4)*a + np.round(jj,4)*b)/ \
                                (np.round(kk,4)*c + np.round(nn,4)*d)
                            if regression == 'On':
                                regressor = LinearRegression()
                                regressor.fit(np.array(x).reshape(-1,1), dataset_test.DE.values)
                                pred = regressor.predict(np.array(x).reshape(-1,1))
                                rmse, r2, cls_acs = target_func(pred=pred, actu=actu)
                                temp_rmse.append(rmse)
                                temp_r2.append(r2)
                                temp_cls_acs.append(cls_acs)
                                value_lst.append([np.round(ii,4), np.round(jj,4), 
                                                  np.round(kk,4), np.round(nn,4)])
                                slops.append(regressor.coef_)
                                intercepts.append(regressor.intercept_)
                            else:                            
                                pred = (coeff*x)+intercept
                                rmse, r2, cls_acs = target_func(pred=pred, actu=actu)
                                temp_rmse.append(rmse)
                                temp_r2.append(r2)
                                temp_cls_acs.append(cls_acs)
                                value_lst.append([np.round(ii,4), np.round(jj,4), 
                                                  np.round(kk,4), np.round(nn,4)])
        print(value_lst[args], temp_rmse[args], temp_r2[args], temp_cls_acs[args])    
    
    print('------------------------------------',len(temp_rmse))
    
    new_rmse.append(temp_rmse[args])
    new_r2.append(temp_r2[args])
    new_cls.append(temp_cls_acs[args])
    coeff_list.append(value_lst[args])
    min_slops.append(slops[args])
    min_intercepts.append(intercepts[args])

origional eqution:  (rp_B + sqrt(fabs(rd_A)))/(rp_B**3 + rp_A**3)
operation list   :  [1, 'sqrt', 3, 3]
sign list        :  ['+', '+', '+', '+']
feature list     :  ['rp_B', 'rd_A', 'rp_B', 'rp_A']
rmse:  0.096339975803 r2 score:  0.9507837413131128 class accuracy:  93.90243902439025


/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/n

/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/n

/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/n

/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/n

...

/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/n

/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/n

/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/n

/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/uday/anaconda3/lib/python3.7/site-packages/n

[-0.9, -0.2, -0.8, -1.0] 0.104711522962 0.9413351499352911 93.90243902439025
[-0.9, -0.2, -0.8, -1.0] 0.104711522962 0.9413351499352911 93.90243902439025
[-0.96, -0.2, -0.76, -0.98] 0.104684982763 0.9413666280122461 93.90243902439025
[-0.968, -0.2, -0.758, -0.98] 0.104684623454 0.9413670541044952 93.90243902439025
------------------------------------ 265011


In [9]:
df = pd.DataFrame()
df['Equations'] = new_equation_list
df['old_rmse_for_full_dataset']=old_rmse
df['new_rmse']=new_rmse
df['old_r2'] = old_r2
df['new_r2'] = new_r2
df['old_class_accuracy']= old_cls
df['new_class_accuracy']= new_cls
df['individual_coeff'] = coeff_list
df['slops'] = [round(num[0], 3) for num in min_slops]
df['intecepts'] = min_intercepts
#df['classification accuracy'] = temp

In [10]:
df.to_excel('optimization_using_r2_score.xlsx')

In [11]:
test_func = 0.633*(-0.85*dataset_test['rp_B'].values**2+1.09*np.exp(dataset_test['rs_B'].values))/dataset_test['rp_A'].values**2  -0.341319

In [12]:
pred_species = ['RS' if i < 0 else 'ZB' for i in test_func]
quant_res    = ([i for i, j in zip(actute_species, pred_species) if i == j])

In [13]:
df

,Equations,old_rmse_for_full_dataset,new_rmse,old_r2,new_r2,old_class_accuracy,new_class_accuracy,individual_coeff,slops,intecepts
0,(rp_B + sqrt(fabs(rd_A)))/(rp_B**3 + rp_A**3),0.096340,0.096113,0.950784,0.951026,93.902439,93.902439,"[-0.642, -0.502, -1.17, -1.11]",1.604,-0.308942
1,(rs_B**2 + sqrt(fabs(rd_A)))/(rp_B**3 + rp_A**3),0.096852,0.096719,0.950233,0.950377,92.682927,93.902439,"[-1.064, -0.924, -1.01, -0.99]",0.909,-0.307460
2,(rs_B + sqrt(fabs(rd_A)))/(rp_A**3 + rp_B**3),0.099324,0.096275,0.947525,0.950853,91.463415,98.780488,"[-1.056, -0.38, -0.968, -1.068]",1.312,-0.296367
3,(rp_B + sqrt(fabs(rp_A)))/(rp_A**3 + rp_B**3),0.102806,0.102256,0.943570,0.944206,97.560976,96.341463,"[-0.6, -1.014, -0.71, -0.61]",0.826,-0.344601
4,(rs_B + sqrt(fabs(rp_A)))/(rp_B**3 + rp_A**3),0.102875,0.101052,0.943490,0.945583,96.341463,98.780488,"[-1.064, -0.4, -0.832, -0.732]",1.168,-0.323984
5,(rp_A + exp(rs_B))/(rp_A**3 + rp_B**3),0.103255,0.102543,0.943048,0.943875,96.341463,96.341463,"[-0.65, -0.826, -1.184, -1.13]",0.881,-0.340435
6,(rp_B + sqrt(fabs(rs_A)))/(rp_A**3 + rp_B**3),0.103570,0.102925,0.942680,0.943432,98.780488,98.780488,"[-1.15, -1.21, -0.878, -0.958]",0.797,-0.332789
7,(rs_B + sqrt(fabs(rs_A)))/(rp_B**3 + rp_A**3),0.103946,0.101618,0.942239,0.944938,97.560976,98.780488,"[-0.864, -0.202, -0.846, -0.646]",1.551,-0.315048
8,(rs_A + exp(rs_B))/(rp_B**3 + rp_A**3),0.104431,0.103447,0.941667,0.942823,97.560976,98.780488,"[-0.078, 0.95, -1.15, -0.848]",-0.850,-0.290679
9,(rs_B**2 + sqrt(fabs(rp_A)))/(rp_B**3 + rp_A**3),0.104959,0.103101,0.941041,0.943228,97.560976,96.341463,"[-0.418, -0.996, -0.678, -0.898]",1.217,-0.344039
